In [1]:
!nvidia-smi

Sat May  1 07:05:18 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8     7W /  75W |      0MiB /  7611MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
 0ae3c26e-76cc-4d07-85f0-411498afdb72.jpg
 0eafa508-4574-4058-8434-df3bb8773faa.jpg
 11124453_827213970666093_1355027621_n.jpg
 11173623_827213943999429_1223020897_n.jpg
 11173694_827213907332766_93223594_n.jpg
 11178426_827213843999439_509973941_n.jpg
 11185820_827213857332771_1539925406_n.jpg
 11198979_827213827332774_1037022449_n.jpg
 11845964_912720492133682_503529493_n.jpg
 11872978_912720495467015_273847539_n.jpg
 12067323_898162850274051_1280517993_n.jpg
 12200457_1659484477627286_1223599808_n.jpg
 20151111_220551.jpg
 20160923_150323.jpg
 2fd0249e-9230-4aed-b9c5-f3e3c04ad037.jpg
 4226acd4-8322-465b-8cf8-29ccf72dc1a1.jpg
 49a81be7-6beb-4535-b8f8-ef3843433826.jpg
 5ad14af1-3b50-4ea9-bfd0-c7562467c021.jpg
 5e00bc86-b58c-4762-99e7-8c266f58ff47.jpg
 63c94ba4-4564-4cfb-ba3a-6d69f0a2d967.jpg
'7390acab-fb55-4008-bcb8-9f2a14b85894 (1).mp4'
 7390acab-fb55-4008-bcb8-9f2a14b85894.mp4
 79ec4b80-bfee-4f44-aa17-a627b499c0bd.jpg
'Aadhar card'
'aadhar card.pdf'
 ab5f3

In [3]:
# Clone
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 14997, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 14997 (delta 18), reused 31 (delta 12), pack-reused 14951
Receiving objects: 100% (14997/14997), 13.38 MiB | 20.63 MiB/s, done.
Resolving deltas: 100% (10180/10180), done.


In [4]:

# Configure
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile

/content/darknet


In [5]:

# Compile
!make

mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:926:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1127:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
             ^~
./src/image_

In [6]:
# Make a copy of yolov3.cfg
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [7]:
# Change lines in yolov3.cfg file
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=2@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=2@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=2@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=21@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=21@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=21@' cfg/yolov3_training.cfg

In [8]:
!echo -e 'Person with mask\Person with no mask' > data/obj.names
!echo -e 'classes= 2\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data

In [9]:
!mkdir data/obj
!unzip /mydrive/yolov3/images.zip -d data/obj

Archive:  /mydrive/yolov3/images.zip
   creating: data/obj/images/
  inflating: data/obj/images/0.jpg   
  inflating: data/obj/images/0.txt   
  inflating: data/obj/images/1.jpg   
  inflating: data/obj/images/1.txt   
  inflating: data/obj/images/10.jpg  
  inflating: data/obj/images/10.txt  
  inflating: data/obj/images/100.jpeg  
  inflating: data/obj/images/100.txt  
  inflating: data/obj/images/101.jpeg  
  inflating: data/obj/images/101.txt  
  inflating: data/obj/images/102.jpeg  
 extracting: data/obj/images/102.txt  
  inflating: data/obj/images/103.jpg  
 extracting: data/obj/images/103.txt  
  inflating: data/obj/images/104.jpg  
 extracting: data/obj/images/104.txt  
 extracting: data/obj/images/105.jpg  
  inflating: data/obj/images/105.txt  
  inflating: data/obj/images/106.jpg  
  inflating: data/obj/images/106.txt  
 extracting: data/obj/images/107.jpg  
  inflating: data/obj/images/107.txt  
 extracting: data/obj/images/108.png  
  inflating: data/obj/images/108.txt  


In [10]:
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2021-05-01 07:09:46--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  64.9MB/s    in 2.4s    

2021-05-01 07:09:49 (64.9 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



In [11]:
!./darknet detector train data/obj.data cfg/yolov3_training.cfg /mydrive/yolov3/yolov3_training_last.weights -dont_show

Streaming output truncated to the last 5000 lines.
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.859736), count: 3, class_loss = 0.000057, iou_loss = 0.029276, total_loss = 0.029333 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.879129), count: 14, class_loss = 0.000011, iou_loss = 0.184867, total_loss = 0.184877 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
 total_bbox = 378398, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.851448), count: 5, class_loss = 0.468565, iou_loss = 0.067367, total_loss = 0.535931 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.858612), count: 3, class_loss = 0.007968, iou_loss = 0.056646, total_loss = 0.064614 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.